In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Setup

In [2]:
!git clone https://github.com/neuspell/neuspell
%cd neuspell

Cloning into 'neuspell'...
remote: Enumerating objects: 653, done.
remote: Counting objects: 100% (456/456), done.
remote: Compressing objects: 100% (287/287), done.
remote: Total 653 (delta 280), reused 303 (delta 151), pack-reused 197
Receiving objects: 100% (653/653), 74.43 MiB | 25.76 MiB/s, done.
Resolving deltas: 100% (306/306), done.
/content/neuspell


In [3]:
!pip install -e .

Obtaining file:///content/neuspell
     |████████████████████████████████| 2.1MB 12.8MB/s 
     |████████████████████████████████| 1.2MB 50.9MB/s 
     |████████████████████████████████| 133kB 51.4MB/s 
     |████████████████████████████████| 901kB 45.9MB/s 
     |████████████████████████████████| 3.3MB 55.9MB/s 
     |████████████████████████████████| 133kB 64.4MB/s 
     |████████████████████████████████| 81kB 10.1MB/s 
     |████████████████████████████████| 7.5MB 52.0MB/s 
ERROR: botocore 1.20.62 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.
  Running setup.py develop for neuspell


In [4]:
!pip install urllib3==1.25.4

     |████████████████████████████████| 133kB 14.0MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [5]:
!pip install folium==0.2.1

     |████████████████████████████████| 71kB 6.7MB/s 
  Created wheel for folium: filename=folium-0.2.1-cp37-none-any.whl size=79979 sha256=83e6fa20c3be3e8ad656c6fd35a9fb4794e9c3458744c73b838f9be68f061f77
  Stored in directory: /root/.cache/pip/wheels/b8/09/f0/52d2ef419c2aaf4fb149f92a33e0008bdce7ae816f0dd8f0c5
Successfully built folium
  Found existing installation: folium 0.8.3
    Uninstalling folium-0.8.3:
      Successfully uninstalled folium-0.8.3


In [6]:
!pip install -r extras-requirements.txt

Obtaining file:///content/neuspell (from -r extras-requirements.txt (line 1))
Obtaining file:///content/neuspell (from -r extras-requirements.txt (line 2))
     |████████████████████████████████| 522kB 12.6MB/s 
     |████████████████████████████████| 266kB 29.8MB/s 
     |████████████████████████████████| 122kB 32.0MB/s 
  Created wheel for jsonnet: filename=jsonnet-0.17.0-cp37-cp37m-linux_x86_64.whl size=3388824 sha256=a898d7cd0653497175b9f78576aead0c89bc8bc9fc033d43eaac9f0c28a3a277
  Stored in directory: /root/.cache/pip/wheels/26/7a/37/7dbcc30a6b4efd17b91ad1f0128b7bbf84813bd4e1cfb8c1e3
  Created wheel for overrides: filename=overrides-3.1.0-cp37-none-any.whl size=10174 sha256=262e60afe513a4ec3e8ec6490d64e6b0953d1fbd4cb45d173765d04d8d1b0948
  Stored in directory: /root/.cache/pip/wheels/5c/24/13/6ef8600e6f147c95e595f1289a86a3cc82ed65df57582c65a9
Successfully built jsonnet overrides
ERROR: allennlp 1.5.0 has requirement torch<1.8.0,>=1.6.0, but you'll have torch 1.8.1+cu101 which is 

In [7]:
!pip install torch==1.6.0

     |████████████████████████████████| 748.8MB 23kB/s 
ERROR: torchvision 0.9.1+cu101 has requirement torch==1.8.1, but you'll have torch 1.6.0 which is incompatible.
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.6.0 which is incompatible.
ERROR: allennlp 1.5.0 has requirement transformers<4.3,>=4.1, but you'll have transformers 4.5.1 which is incompatible.
  Found existing installation: torch 1.8.1+cu101
    Uninstalling torch-1.8.1+cu101:
      Successfully uninstalled torch-1.8.1+cu101


In [8]:
!pip install transformers==4.1

     |████████████████████████████████| 1.5MB 13.1MB/s 
     |████████████████████████████████| 2.9MB 51.4MB/s 
  Found existing installation: tokenizers 0.10.2
    Uninstalling tokenizers-0.10.2:
      Successfully uninstalled tokenizers-0.10.2
  Found existing installation: transformers 4.5.1
    Uninstalling transformers-4.5.1:
      Successfully uninstalled transformers-4.5.1


In [9]:
import neuspell

data folder is set to `/content/neuspell/neuspell/../data` script


In [10]:
%cd data/traintest
!python download_datafiles.py 

/content/neuspell/data/traintest
./wo_context created


In [11]:
%cd /content/neuspell

/content/neuspell


# Library

In [12]:
import torch
from torch import nn
from torch.nn.utils.rnn import pad_sequence
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch.nn.functional as F
import numpy as np

import time

from neuspell.seq_modeling.subwordbert import load_model
from neuspell.seq_modeling.helpers import load_data, train_validation_split, batch_accuracy_func
from neuspell.seq_modeling.helpers import get_tokens, progressBar
from neuspell.seq_modeling.helpers import batch_iter, labelize, tokenize, bert_tokenize_for_valid_examples

from neuspell.seq_modeling.helpers import load_vocab_dict, save_vocab_dict

# Train

## Load dataset, vocab

In [13]:
train_data = load_data('/content/neuspell/data/traintest/','test.1blm','test.1blm.noise.prob')

train_data, valid_data = train_validation_split(train_data, 0.90, seed=1)

vocab_ref = {}

vocab = get_tokens([i[0] for i in train_data],
                           keep_simple=True,
                           min_max_freq=(2,float("inf")),
                           topk=100000,
                           intersect=vocab_ref,
                           load_char_tokens=True)

# save_vocab_dict('/content/drive/MyDrive/NLP/bert_vocab.pkl', vocab)

273134it [00:00, 324564.91it/s]
273134it [00:00, 1248404.39it/s]
  0%|          | 0/245821 [00:00<?, ?it/s]

loaded tuples of (corr,incorr) examples from /content/neuspell/data/traintest/


100%|██████████| 245821/245821 [00:01<00:00, 139424.34it/s]


Total tokens found: 155272


  0%|          | 0/245821 [00:00<?, ?it/s]

Total tokens retained: 139243
Total tokens retained: 67965
Total tokens retained: 67965
loading character tokens


100%|██████████| 245821/245821 [00:01<00:00, 184178.27it/s]

number of unique chars found: 98
{'<<CHAR_UNK>>': 0, '<<CHAR_PAD>>': 1, '<<CHAR_START>>': 2, '<<CHAR_END>>': 3, 'H': 4, 'e': 5, ' ': 6, 'w': 7, 'a': 8, 's': 9, 'b': 10, 'r': 11, 'i': 12, 'f': 13, 'l': 14, 'y': 15, 'h': 16, 'o': 17, 'p': 18, 't': 19, 'z': 20, 'd': 21, 'n': 22, 'c': 23, 'k': 24, 'j': 25, 'u': 26, '.': 27, 'A': 28, 'm': 29, 'B': 30, 'g': 31, 'G': 32, 'M': 33, '3': 34, 'v': 35, ',': 36, '7': 37, '6': 38, '2': 39, 'T': 40, 'D': 41, 'S': 42, 'E': 43, '1': 44, '-': 45, 'x': 46, '4': 47, '9': 48, '(': 49, 'U': 50, 'P': 51, 'I': 52, ')': 53, "'": 54, ';': 55, 'q': 56, '"': 57, '?': 58, 'Y': 59, 'C': 60, 'O': 61, 'K': 62, 'L': 63, 'N': 64, 'R': 65, 'W': 66, '0': 67, '8': 68, 'J': 69, ':': 70, 'F': 71, '5': 72, '%': 73, 'V': 74, '/': 75, '$': 76, 'Z': 77, 'Q': 78, '!': 79, '[': 80, ']': 81, 'X': 82, '&': 83, '@': 84, '+': 85, '_': 86, '#': 87, '*': 88, '|': 89, '>': 90, '^': 91, '\\': 92, '=': 93, '<': 94, '~': 95, '{': 96, '}': 97}


## Load model

In [31]:
model = load_model(vocab,"bert-base-cased")
model = model.cuda()


VALID_BATCH_SIZE = 32

data_iter = batch_iter(train_data, batch_size=VALID_BATCH_SIZE, shuffle=False)

TRAIN_BATCH_SIZE = 32

DEVICE = 'cuda'


Number of parameters in the model: 160577664


## freeze layers

In [32]:
num_freeze=2
for layers in model.bert_model.encoder.layer[:num_freeze]:
    for param in layers.parameters():
        param.requires_grad = False

## Bert optimizer

In [33]:
START_EPOCH = 1
N_EPOCHS = 5

GRADIENT_ACC = 4
max_dev_acc, argmax_dev_acc = -1, -1

In [34]:
# from pytorch_pretrained_bert import BertAdam

# param_optimizer = list(model.named_parameters())
# no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
# optimizer_grouped_parameters = [
#     {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
#     {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
# ]
# t_total = int(len(train_data) / TRAIN_BATCH_SIZE / GRADIENT_ACC * N_EPOCHS)
# optimizer = BertAdam(optimizer_grouped_parameters,lr=5e-5,warmup=0.1,t_total=t_total)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)



## Set epoch, start training

In [35]:
# train and eval
for epoch_id in range(START_EPOCH,N_EPOCHS+1):

    print(f"In epoch: {epoch_id}")

    # train loss and backprop
    train_loss = 0.
    train_acc = 0.
    train_acc_count = 0.
    print("train_data size: {}".format(len(train_data)))
    
    train_data_iter = batch_iter(train_data, batch_size=TRAIN_BATCH_SIZE, shuffle=True)
    nbatches = int(np.ceil(len(train_data)/TRAIN_BATCH_SIZE))
    optimizer.zero_grad()

    for batch_id, (batch_labels,batch_sentences) in enumerate(train_data_iter):
        optimizer.zero_grad()
        st_time = time.time()

        # set batch data for bert
        batch_labels_, batch_sentences_, batch_bert_inp, batch_bert_splits = bert_tokenize_for_valid_examples(batch_labels,batch_sentences)                
        if len(batch_labels_)==0:
            print("################")
            print("Not training the following lines due to pre-processing mismatch: \n")
            print([(a,b) for a,b in zip(batch_labels,batch_sentences)])
            print("################")
            continue
        else:
            batch_labels, batch_sentences = batch_labels_, batch_sentences_

        batch_bert_inp = {k:v.to(DEVICE) for k,v in batch_bert_inp.items()}

        # set batch data for others
        batch_labels, batch_lengths = labelize(batch_labels, vocab)
        batch_lengths = batch_lengths.to(DEVICE)
        batch_labels = batch_labels.to(DEVICE)

        # forward
        model.train()
        
        loss = model(batch_bert_inp, batch_bert_splits, targets=batch_labels)
        
        batch_loss = loss.cpu().detach().numpy()
        train_loss += batch_loss

        # backward
        # if GRADIENT_ACC > 1:
        #     loss = loss / GRADIENT_ACC
        loss.backward()
        # step
        # if (batch_id + 1) % GRADIENT_ACC == 0 or batch_id >= nbatches - 1:
            # torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
            # scheduler.step()
        # optimizer.zero_grad()

        # compute accuracy in numpy
        if batch_id%10000==0:

            train_acc_count += 1

            model.eval()
            with torch.no_grad():
                _, batch_predictions = model(batch_bert_inp, batch_bert_splits, targets=batch_labels)

            model.train()

            batch_labels = batch_labels.cpu().detach().numpy()
            batch_lengths = batch_lengths.cpu().detach().numpy()
            ncorr,ntotal = batch_accuracy_func(batch_predictions,batch_labels,batch_lengths)
            batch_acc = ncorr/ntotal
            train_acc += batch_acc     

        # update progress
        progressBar(batch_id+1,
                    int(np.ceil(len(train_data) / TRAIN_BATCH_SIZE)), 
                    ["batch_time","batch_loss","avg_batch_loss","batch_acc","avg_batch_acc"],
                    [time.time()-st_time,batch_loss,train_loss/(batch_id+1),batch_acc,train_acc/train_acc_count]) 
    
    print(f"\nEpoch {epoch_id} train_loss: {train_loss/(batch_id+1)}")

    # save model every epoch
    model_name = "bert_epoch_" + str(epoch_id) + '.pt'
    # torch.save(model.state_dict(), 
    #         '/content/drive/MyDrive/NLP/'+model_name)

    # valid loss
    valid_loss = 0.
    valid_acc = 0.
    print("valid_data size: {}".format(len(valid_data)))

    valid_data_iter = batch_iter(valid_data, batch_size=VALID_BATCH_SIZE, shuffle=False)

    for batch_id, (batch_labels,batch_sentences) in enumerate(valid_data_iter):

        st_time = time.time()
        # set batch data for bert
        # batch_labels_, batch_sentences_, batch_bert_inp, batch_bert_splits = bert_tokenize_for_valid_examples(batch_labels,batch_sentences)

        batch_labels, batch_sentences, batch_bert_inp, batch_bert_splits = bert_tokenize_for_valid_examples(batch_labels,batch_sentences)
        """
        if len(batch_labels_)==0:
            print("################")
            print("Not validating the following lines due to pre-processing mismatch: \n")
            print([(a,b) for a,b in zip(batch_labels,batch_sentences)])
            print("################")
            continue
        else:
        
            batch_labels, batch_sentences = batch_labels_, batch_sentences_
        """

        batch_bert_inp = {k:v.to(DEVICE) for k,v in batch_bert_inp.items()}


        # set batch data for others
        batch_labels, batch_lengths = labelize(batch_labels, vocab)
        batch_lengths = batch_lengths.to(DEVICE)
        batch_labels = batch_labels.to(DEVICE)

        # forward
        model.eval()
        with torch.no_grad():
            batch_loss, batch_predictions = model(batch_bert_inp, batch_bert_splits, targets=batch_labels)
        model.train()        
        valid_loss += batch_loss
        # compute accuracy in numpy
        batch_labels = batch_labels.cpu().detach().numpy()
        batch_lengths = batch_lengths.cpu().detach().numpy()
        ncorr,ntotal = batch_accuracy_func(batch_predictions,batch_labels,batch_lengths)
        batch_acc = ncorr/ntotal
        valid_acc += batch_acc
        # update progress
        progressBar(batch_id+1,
                    int(np.ceil(len(valid_data) / VALID_BATCH_SIZE)), 
                    ["batch_time","batch_loss","avg_batch_loss","batch_acc","avg_batch_acc"], 
                    [time.time()-st_time,batch_loss,valid_loss/(batch_id+1),batch_acc,valid_acc/(batch_id+1)])

    print(f"\nEpoch {epoch_id} valid_loss: {valid_loss/(batch_id+1)}")
    torch.save({
            'epoch': epoch_id,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': valid_loss}, f'/content/gdrive/MyDrive/freeze{num_freeze}epoch:{epoch_id}valid_acc{valid_acc/(batch_id+1)}.hdf5')

  

In epoch: 1
train_data size: 245821
Percent: [----------------------------->] 100% || batch_time: 0.2483 || batch_loss: 0.5298 || avg_batch_loss: 1.1982 || batch_acc: 0.0307 || avg_batch_acc: 0.0307 
Epoch 1 train_loss: 1.198212431657907
valid_data size: 27313
Percent: [----------------------------->] 100% || batch_time: 0.0656 || batch_loss: 0.3782 || avg_batch_loss: 0.4170 || batch_acc: 0.9522 || avg_batch_acc: 0.9368 
Epoch 1 valid_loss: 0.41701632518260207
In epoch: 2
train_data size: 245821
Percent: [----------------------------->] 100% || batch_time: 0.2125 || batch_loss: 0.2363 || avg_batch_loss: 0.3284 || batch_acc: 0.9454 || avg_batch_acc: 0.9454 
Epoch 2 train_loss: 0.3284163758705693
valid_data size: 27313
Percent: [----------------------------->] 100% || batch_time: 0.0624 || batch_loss: 0.2727 || avg_batch_loss: 0.2355 || batch_acc: 0.9636 || avg_batch_acc: 0.9567 
Epoch 2 valid_loss: 0.23552788285412052
In epoch: 3
train_data size: 245821
Percent: [-----------------------

## Evaluation

In [38]:
checkpoint = torch.load("/content/gdrive/MyDrive/freeze6epoch:4valid_acc0.9674749775531915.hdf5",map_location='cpu')
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [39]:
from neuspell.seq_modeling.subwordbert import model_inference

test_data = load_data('/content/neuspell/data/traintest/','test.bea60k','test.bea60k.noise')

predicted_result = model_inference(model, test_data, 1, 'cuda', 16, vocab)

63044it [00:00, 428885.83it/s]
63044it [00:00, 1229400.94it/s]
1it [00:00,  5.90it/s]

loaded tuples of (corr,incorr) examples from /content/neuspell/data/traintest/
###############################################
data size: 63044


3941it [03:35, 18.27it/s]


Epoch None valid_loss: 0.2912500051796187
total inference time for this data is: 215.668664 secs
###############################################


total token count: 1059069
_corr2corr:964166, _corr2incorr:24893, _incorr2corr:48199, _incorr2incorr:21811
accuracy is 0.955900890310263
word correction rate is 0.6884587916011998
###############################################


In [40]:

test_data = load_data('/content/neuspell/data/traintest/','test.jfleg','test.jfleg.noise')

predicted_result = model_inference(model, test_data, 1, 'cuda', 16, vocab)

1601it [00:00, 350309.39it/s]
1601it [00:00, 1581880.02it/s]
2it [00:00, 16.78it/s]

loaded tuples of (corr,incorr) examples from /content/neuspell/data/traintest/
###############################################
data size: 1601


101it [00:06, 15.63it/s]


Epoch None valid_loss: 0.17520444473196375
total inference time for this data is: 6.473592 secs
###############################################


total token count: 33473
_corr2corr:31047, _corr2incorr:416, _incorr2corr:1562, _incorr2incorr:448
accuracy is 0.974188151644609
word correction rate is 0.7771144278606965
###############################################
